Following : https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
Training ResNet50 with freezing all layers except for the last block

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.applications import imagenet_utils
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow does not work on Google Colab notebooks, --> use cv2_imshow instead

In [ ]:
# loading settings
model_string = "resnet50"  # inceptionv3, resnet50, vgg16
dataset_string = "imagenet"  # imagenet

# other settings
heatmap_intensity = 0.5
#path_to_dataset = '/content/gdrive/My Drive/What are CNNs looking at/files/New Masks Dataset/'
path_to_dataset = '/content/gdrive/My Drive/Kaggle/files/Face Mask Dataset'
path_to_dataset = '/content/gdrive/.shortcut-targets-by-id/1wI0fw-edJDu1RgH2Ldk3n4VBJ4m4EJuv/combined'
path_to_dataset= '/content/gdrive/MyDrive/combined.rar (Unzipped Files)/combined'
base_learning_rate = 0.001

In [ ]:
def gradCAM(orig, model, model_string, DIM, HM_DIM, last_layer, classes, intensity=0.5, res=250):

    if model_string == "inceptionv3":
        from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

    elif model_string == "resnet50":
        from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

    elif model_string == "vgg16":
        from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

    else:  # use InceptionV3 and imagenet as default
        from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions


    img = image.load_img(orig, target_size=(DIM, DIM))

    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    preds = model.predict(x)
    for pred_it, label_it in zip(preds[0], classes):
      print(label_it, ": ", pred_it)
    print("preds")
    print(preds)
    prob = np.max(preds[0])
    index = list(preds[0]).index(prob)
    label = classes[index]
    label = "{}: {:.2f}%".format(label, prob * 100)
    print("[INFO] {}".format(label))

    with tf.GradientTape() as tape:
        last_conv_layer = model.get_layer(last_layer)
        iterate = tf.keras.models.Model([model.inputs], [model.output, last_conv_layer.output]) # run model and achive certain output 'last_conv_layer.output'
        model_out, last_conv_layer = iterate(x)
        class_out = model_out[:, np.argmax(model_out[0])]
        grads = tape.gradient(class_out, last_conv_layer) # class out: take derivative; last_conv_layer: variable to derive from
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    heatmap = heatmap.reshape((HM_DIM, HM_DIM))

    img_original = cv2.imread(orig)
    heatmap = cv2.resize(heatmap, (img_original.shape[1], img_original.shape[0]))
    # plt.matshow(heatmap)
    # plt.show()

    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    # plt.matshow(heatmap)
    # plt.show()
    img_heatmap = heatmap * intensity + img_original

    cv2_imshow(img_heatmap)

In [ ]:
# model dependent imports, weight loading and model dependent parameter setting
K.clear_session()
if model_string == "inceptionv3":
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    Model = InceptionV3
    if dataset_string == "imagenet":
        model = InceptionV3(weights='imagenet')
    target_input_dimension = 299
    heatmap_dimension = 8
    last_layer_name = 'conv2d_93'
elif model_string == "resnet50":
    from tensorflow.keras.applications import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
    Model = ResNet50
    if dataset_string == "imagenet":
        model = ResNet50(weights="imagenet")
    target_input_dimension = 224
    heatmap_dimension = 7
    last_layer_name = 'conv5_block3_3_conv'
elif model_string == "vgg16":
    from tensorflow.keras.applications import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
    Model = VGG16
    if dataset_string == "imagenet":
        model = VGG16(weights="imagenet")
    target_input_dimension = 224
    heatmap_dimension = 14
    last_layer_name = 'block5_conv3'
else:  # use InceptionV3 and imagenet as default
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
    if dataset_string == "imagenet":
        model = InceptionV3(weights='imagenet')
    target_input_dimension = 299
    heatmap_dimension = 8
    last_layer_name = 'conv2d_93'

In [ ]:
# mount google drive to access database

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
#%cd /content/gdrive/Computers/My Laptop/combined
#%cd /content/gdrive/.shortcut-targets-by-id/1wI0fw-edJDu1RgH2Ldk3n4VBJ4m4EJuv/combined
#!ls -la gdrive/MyDrive/
%cd /content/gdrive/MyDrive/combined.rar (Unzipped Files)/combined



In [ ]:
# import database

train_dir = os.path.join(path_to_dataset, 'Train')
validation_dir = os.path.join(path_to_dataset, 'Validation')
test_dir = os.path.join(path_to_dataset, 'Test')
BATCH_SIZE = 64
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)
validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)
test_dataset = image_dataset_from_directory(test_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

In [ ]:
class_names = train_dataset.class_names
print(class_names)

In [ ]:
# training the last block of Resnet50 since we are using a different dataset wih imagenet weights
# creating the model

# currently ResNet50 is hard coded
K.clear_session()

# load ResNet50 without dense layers
base_model = ResNet50(input_shape=(target_input_dimension, 
                               target_input_dimension, 
                               3), 
                  include_top=False, 
                  weights='imagenet')
for layer in base_model.layers[:81]:
  layer.trainable = False
# printing the layers that are trainable
for i, layer in enumerate(base_model.layers):
  print(i, layer.name, "-", layer.trainable)

inputs = tf.keras.Input(shape=(target_input_dimension, 
                               target_input_dimension, 
                               3))
x = preprocess_input(inputs)

In [ ]:
to_res = (224,224)
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Lambda(lambda image: tf.image.resize(image, to_res))) 
inputs = tf.keras.Input(shape=(target_input_dimension, 
                               target_input_dimension, 
                               3), name="model_input")
x = preprocess_input(inputs)
x = base_model(x)  # Why training=False?
x = tf.identity(x) # needed to be able to obtain heatmap
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=x)
  

#check_point = tf.keras.callbacks.ModelCheckpoint(filepath="cifar10.h5",
 #                                         monitor="val_acc",
  #                                        mode="max",
   #                                       save_best_only=True,
    #                                     )

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
#history = model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=1,
#                    validation_data=(x_test, y_test),
#                    callbacks=[check_point])
history = model.fit(train_dataset,
                    epochs=10,
                    validation_data=validation_dataset)
model.summary()
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_dataset)
%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
!mkdir -p saved_model
model.save('saved_model/new_resnet50trained_wrongmask')

# heatmaps
%cd /content/gdrive/.shortcut-targets-by-id/1wI0fw-edJDu1RgH2Ldk3n4VBJ4m4EJuv/combined
images = ['Test/Mask/45.png','Test/Mask/2086.jpg','Test/Mask/2267.png','Test/Mask/2190.png', 'Test/Non Mask/45.png', 'Test/Non Mask/5878.png','Test/Non Mask/real_01047.jpg', 'Test/Non Mask/real_01072.jpg']

DIM = 224
for orig in images:
  gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_dataset)
%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
images = ['maskWrong.PNG','nomask.PNG','abin_halfmask.jpg', 'maskdown.PNG', 'maskdown2.PNG','maskdown3.PNG','maskchin.PNG','maskchin1.PNG']


DIM = 224
for orig in images:
  gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)



In [ ]:
history2 = model.fit(train_dataset,
                    epochs=6,
                    validation_data=validation_dataset)


In [ ]:
#%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
images = ['maskWrong.PNG','nomask.PNG','abin_halfmask.jpg', 'maskdown.PNG', 'maskdown2.PNG','maskdown3.PNG','maskchin.PNG','maskchin1.PNG']

DIM = 224
for orig in images:
  gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity_1',class_names)

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_dataset)

In [ ]:
history2 = model.fit(train_dataset,
                    epochs=2,
                    validation_data=validation_dataset)

In [ ]:
history2 = model.fit(train_dataset,
                    epochs=4,
                    validation_data=validation_dataset)

In [ ]:
orig ='maskWrong.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='nomask.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='abin_halfmask.jpg'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)

In [ ]:
orig ='maskchin.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='abin_nomask.jpg'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='SBhand.jpg'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)

In [ ]:
%cd /content/gdrive/.shortcut-targets-by-id/1D594PlXLC7S5YLnKD2s09vbezmGes8H6/What are CNNs looking at
orig ='maskchin1.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='maskdown.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='maskdown2.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig ='maskdown3.PNG'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
orig =' JLM_3a.jpg'
#orig = 'files/12k Face Mask Dataset/Test/WithoutMask/1439.png'
DIM = 224
gradCAM(orig, model, model_string, target_input_dimension, heatmap_dimension,'tf.identity',class_names)
